### Coursera Capstone Assignment PART 1

Assignment of data scraping neighborhood

##### Importing libraries

In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

##### Getting Wikipedia page

In [0]:
web_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
web_text= requests.get(web_link).text
canada = BeautifulSoup(web_text, 'lxml')

##### Creating Table

In [0]:
canada_columns = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = canada_columns)
data = canada.find('div', class_='mw-parser-output')
table = data.table.tbody
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
postcode = 0
borough = 0
neighborhood = 0

##### Scraping data from wiki to table + combining reuqired rows

In [0]:
#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

##### Ignoring cells with unassigned boroughs + copyinh borough name to neighborhood

In [5]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1                               
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head(10)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


##### Printing df shape

In [6]:
df.shape

(103, 3)

### Coursera Capstone Assignment PART 2

In [0]:
#having too many problems with geocoder
coords = pd.read_csv('https://cocl.us/Geospatial_data')
gdf = pd.DataFrame(coords)

In [0]:
merged = pd.concat([df, coords] , axis=1, join='outer', ignore_index=False, keys=None,
          levels=None, names=None, verify_integrity=False, copy=True)

In [9]:
merged.drop(columns=['Postal Code'])

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


### Coursera Capstone Assignment PART 3

#### Explore Neighborhoods in Toronto

In [10]:
import folium
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.740723, -79.424781], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged['Latitude'], merged['Longitude'], merged['Borough'], merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
CLIENT_ID = 'xxxxxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxxxxxxxxxxxxxxx' # your Foursquare Secret
VERSION = '20200101' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xxxxxxxxxxxxxxxxxxxxxxxxxxxxx
CLIENT_SECRET:xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


In [12]:
neighborhood_latitude = merged.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = merged.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = merged.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

print(merged)


Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.
    Postalcode      Borough  ...   Latitude  Longitude
0          M1B  Scarborough  ...  43.806686 -79.194353
1          M1C  Scarborough  ...  43.784535 -79.160497
2          M1E  Scarborough  ...  43.763573 -79.188711
3          M1G  Scarborough  ...  43.770992 -79.216917
4          M1H  Scarborough  ...  43.773136 -79.239476
..         ...          ...  ...        ...        ...
98         M9N         York  ...  43.706876 -79.518188
99         M9P    Etobicoke  ...  43.696319 -79.532242
100        M9R    Etobicoke  ...  43.688905 -79.554724
101        M9V    Etobicoke  ...  43.739416 -79.588437
102        M9W    Etobicoke  ...  43.706748 -79.594054

[103 rows x 6 columns]


In [13]:
import json
import pandas as pd
from pandas.io.json import json_normalize

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            103
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


toronto_venues = getNearbyVenues(names=merged['Neighborhood'], 
                                 latitudes=merged['Latitude'],
                                 longitudes=merged['Longitude']
                                 )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [14]:
#toronto_venues.columns = ['Neighborhood', 'XXX','Latitude','XXX', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
toronto_venues.shape

(2224, 7)

In [15]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",9,9,9,9,9,9
...,...,...,...,...,...,...
Willowdale West,7,7,7,7,7,7
Woburn,3,3,3,3,3,3
"Woodbine Gardens, Parkview Hill",13,13,13,13,13,13


In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


In [17]:
#from google.colab import drive
#import pandas as pd
#drive.mount('/content/gdrive')
#toronto_venues.to_csv('toronto.csv')
#!cp toronto.csv "/content/gdrive/My Drive"

KeyboardInterrupt: ignored

#### Analyze each neighborhood

In [18]:
# one hot encoding

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']])

# add neighborhood column back to dataframe
merged_venues = pd.concat([toronto_onehot, toronto_venues['Neighborhood']] , axis=1)

# move neighborhood column to the first column
fixed_columns = [merged_venues.columns[-1]] + list(merged_venues.columns[:-1])
merged_venues = merged_venues[fixed_columns]

#merged_venues = merged_venues.drop(merged_venues.columns[0], axis=1) 

merged_venues.head()
#toronto_venues.head()

,Neighborhood,Venue Category_Accessories Store,Venue Category_Afghan Restaurant,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,Venue Category_Aquarium,Venue Category_Art Gallery,Venue Category_Arts & Crafts Store,Venue Category_Asian Restaurant,Venue Category_Athletics & Sports,Venue Category_Auto Workshop,Venue Category_BBQ Joint,Venue Category_Baby Store,Venue Category_Bagel Shop,Venue Category_Bakery,Venue Category_Bank,Venue Category_Bar,Venue Category_Baseball Field,Venue Category_Baseball Stadium,Venue Category_Basketball Court,Venue Category_Basketball Stadium,Venue Category_Beach,Venue Category_Bed & Breakfast,Venue Category_Beer Bar,Venue Category_Beer Store,Venue Category_Belgian Restaurant,Venue Category_Bike Shop,Venue Category_Bistro,Venue Category_Boat or Ferry,Venue Category_Bookstore,Venue Category_Boutique,Venue Category_Brazilian Restaurant,Venue Category_Breakfast Spot,Venue Category_Brewery,...,Venue Category_Skating Rink,Venue Category_Smoke Shop,Venue Category_Smoothie Shop,Venue Category_Snack Place,Venue Category_Soccer Field,Venue Category_Southern / Soul Food Restaurant,Venue Category_Spa,Venue Category_Speakeasy,Venue Category_Sporting Goods Shop,Venue Category_Sports Bar,Venue Category_Stadium,Venue Category_Stationery Store,Venue Category_Steakhouse,Venue Category_Strip Club,Venue Category_Supermarket,Venue Category_Supplement Shop,Venue Category_Sushi Restaurant,Venue Category_Swim School,Venue Category_Taco Place,Venue Category_Tailor Shop,Venue Category_Taiwanese Restaurant,Venue Category_Tanning Salon,Venue Category_Tea Room,Venue Category_Thai Restaurant,Venue Category_Theater,Venue Category_Theme Restaurant,Venue Category_Thrift / Vintage Store,Venue Category_Toy / Game Store,Venue Category_Trail,Venue Category_Train Station,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Video Store,Venue Category_Vietnamese Restaurant,Venue Category_Warehouse Store,Venue Category_Wine Bar,Venue Category_Wine Shop,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
hood_grouped = merged_venues.groupby('Neighborhood').mean().reset_index()
hood_grouped.shape

(98, 272)

In [20]:
num_top_venues = 5

for hood in hood_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = hood_grouped[hood_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                        venue  freq
0  Venue Category_Coffee Shop  0.07
1   Venue Category_Steakhouse  0.04
2         Venue Category_Café  0.04
3          Venue Category_Bar  0.04
4        Venue Category_Hotel  0.03


----Agincourt----
                                      venue  freq
0             Venue Category_Breakfast Spot  0.25
1             Venue Category_Clothing Store  0.25
2                     Venue Category_Lounge  0.25
3  Venue Category_Latin American Restaurant  0.25
4        Venue Category_Monument / Landmark  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                                            venue  freq
0                       Venue Category_Playground   0.5
1                             Venue Category_Park   0.5
2                Venue Category_Accessories Store   0.0
3                    Venue Category_Metro Station   0.0
4  Venue Category_Molecular Gastronomy Restaurant   0.0


----Albion Gardens, Beaumo

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = hood_grouped['Neighborhood']

for ind in np.arange(hood_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hood_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(98, 11)

In [23]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

hood_grouped_clustering = hood_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [29]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

x_merged = merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
x_merged = x_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#drop NaN
x_merged.dropna(inplace=True)

x_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353,2.0,Venue Category_Fast Food Restaurant,Venue Category_Yoga Studio,Venue Category_Diner,Venue Category_Farmers Market,Venue Category_Falafel Restaurant,Venue Category_Event Space,Venue Category_Ethiopian Restaurant,Venue Category_Empanada Restaurant,Venue Category_Electronics Store,Venue Category_Eastern European Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497,2.0,Venue Category_Bar,Venue Category_Yoga Studio,Venue Category_Eastern European Restaurant,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Electronics Store,Venue Category_Field
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711,2.0,Venue Category_Rental Car Location,Venue Category_Intersection,Venue Category_Breakfast Spot,Venue Category_Spa,Venue Category_Pizza Place,Venue Category_Mexican Restaurant,Venue Category_Electronics Store,Venue Category_Medical Center,Venue Category_Dog Run,Venue Category_Doner Restaurant
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917,2.0,Venue Category_Coffee Shop,Venue Category_Korean Restaurant,Venue Category_Yoga Studio,Venue Category_Electronics Store,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Eastern European Restaurant,Venue Category_Ethiopian Restaurant
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,2.0,Venue Category_Hakka Restaurant,Venue Category_Bakery,Venue Category_Fried Chicken Joint,Venue Category_Caribbean Restaurant,Venue Category_Thai Restaurant,Venue Category_Athletics & Sports,Venue Category_Gas Station,Venue Category_Bank,Venue Category_Lounge,Venue Category_Electronics Store


In [30]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import math
# create map
map_clusters = folium.Map(location=[43.740723, -79.424781], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(x_merged['Latitude'], x_merged['Longitude'], x_merged['Neighborhood'], x_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
x_merged.loc[x_merged['Cluster Labels'] == 0, x_merged.columns[[1] + list(range(5, x_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,-79.284577,0.0,Venue Category_Park,Venue Category_Playground,Venue Category_Eastern European Restaurant,Venue Category_Discount Store,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Electronics Store
23,North York,-79.400049,0.0,Venue Category_Park,Venue Category_Bank,Venue Category_Convenience Store,Venue Category_Electronics Store,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Eastern European Restaurant,Venue Category_Empanada Restaurant
25,North York,-79.329656,0.0,Venue Category_Food & Drink Shop,Venue Category_Park,Venue Category_Electronics Store,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Eastern European Restaurant,Venue Category_Yoga Studio
30,North York,-79.464763,0.0,Venue Category_Park,Venue Category_Airport,Venue Category_Snack Place,Venue Category_Comfort Food Restaurant,Venue Category_Comic Shop,Venue Category_Fast Food Restaurant,Venue Category_Farmers Market,Venue Category_Falafel Restaurant,Venue Category_Event Space,Venue Category_Ethiopian Restaurant
40,East York,-79.338106,0.0,Venue Category_Park,Venue Category_Convenience Store,Venue Category_Coffee Shop,Venue Category_Intersection,Venue Category_Eastern European Restaurant,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant
44,Central Toronto,-79.388790,0.0,Venue Category_Gym / Fitness Center,Venue Category_Swim School,Venue Category_Bus Line,Venue Category_Park,Venue Category_Construction & Landscaping,Venue Category_Doner Restaurant,Venue Category_Fast Food Restaurant,Venue Category_Farmers Market,Venue Category_Falafel Restaurant,Venue Category_Event Space
50,Downtown Toronto,-79.377529,0.0,Venue Category_Park,Venue Category_Playground,Venue Category_Trail,Venue Category_Dumpling Restaurant,Venue Category_Discount Store,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Eastern European Restaurant
64,Central Toronto,-79.411307,0.0,Venue Category_Trail,Venue Category_Park,Venue Category_Sushi Restaurant,Venue Category_Jewelry Store,Venue Category_Bus Line,Venue Category_Yoga Studio,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore
72,North York,-79.445073,0.0,Venue Category_Park,Venue Category_Playground,Venue Category_Pub,Venue Category_Japanese Restaurant,Venue Category_Bakery,Venue Category_Ethiopian Restaurant,Venue Category_Empanada Restaurant,Venue Category_Event Space,Venue Category_Electronics Store,Venue Category_Eastern European Restaurant
73,York,-79.428191,0.0,Venue Category_Park,Venue Category_Hockey Arena,Venue Category_Trail,Venue Category_Field,Venue Category_Concert Hall,Venue Category_Construction & Landscaping,Venue Category_Fast Food Restaurant,Venue Category_Farmers Market,Venue Category_Falafel Restaurant,Venue Category_Event Space


In [32]:
x_merged.loc[x_merged['Cluster Labels'] == 1, x_merged.columns[[1] + list(range(5, x_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,-79.498509,1.0,Venue Category_Baseball Field,Venue Category_Yoga Studio,Venue Category_Eastern European Restaurant,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Electronics Store,Venue Category_Field
97,North York,-79.532242,1.0,Venue Category_Baseball Field,Venue Category_Yoga Studio,Venue Category_Eastern European Restaurant,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Electronics Store,Venue Category_Field


In [33]:
x_merged.loc[x_merged['Cluster Labels'] == 2, x_merged.columns[[1] + list(range(5, x_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,-79.194353,2.0,Venue Category_Fast Food Restaurant,Venue Category_Yoga Studio,Venue Category_Diner,Venue Category_Farmers Market,Venue Category_Falafel Restaurant,Venue Category_Event Space,Venue Category_Ethiopian Restaurant,Venue Category_Empanada Restaurant,Venue Category_Electronics Store,Venue Category_Eastern European Restaurant
1,Scarborough,-79.160497,2.0,Venue Category_Bar,Venue Category_Yoga Studio,Venue Category_Eastern European Restaurant,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Electronics Store,Venue Category_Field
2,Scarborough,-79.188711,2.0,Venue Category_Rental Car Location,Venue Category_Intersection,Venue Category_Breakfast Spot,Venue Category_Spa,Venue Category_Pizza Place,Venue Category_Mexican Restaurant,Venue Category_Electronics Store,Venue Category_Medical Center,Venue Category_Dog Run,Venue Category_Doner Restaurant
3,Scarborough,-79.216917,2.0,Venue Category_Coffee Shop,Venue Category_Korean Restaurant,Venue Category_Yoga Studio,Venue Category_Electronics Store,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Eastern European Restaurant,Venue Category_Ethiopian Restaurant
4,Scarborough,-79.239476,2.0,Venue Category_Hakka Restaurant,Venue Category_Bakery,Venue Category_Fried Chicken Joint,Venue Category_Caribbean Restaurant,Venue Category_Thai Restaurant,Venue Category_Athletics & Sports,Venue Category_Gas Station,Venue Category_Bank,Venue Category_Lounge,Venue Category_Electronics Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Etobicoke,-79.577201,2.0,Venue Category_Liquor Store,Venue Category_Pharmacy,Venue Category_Beer Store,Venue Category_Pizza Place,Venue Category_Coffee Shop,Venue Category_Café,Venue Category_Falafel Restaurant,Venue Category_Event Space,Venue Category_Ethiopian Restaurant,Venue Category_Farmers Market
99,Etobicoke,-79.532242,2.0,Venue Category_Pizza Place,Venue Category_Discount Store,Venue Category_Middle Eastern Restaurant,Venue Category_Intersection,Venue Category_Coffee Shop,Venue Category_Chinese Restaurant,Venue Category_Sandwich Place,Venue Category_Donut Shop,Venue Category_Dog Run,Venue Category_Doner Restaurant
100,Etobicoke,-79.554724,2.0,Venue Category_Mobile Phone Shop,Venue Category_Pizza Place,Venue Category_Sandwich Place,Venue Category_Bus Line,Venue Category_Drugstore,Venue Category_Discount Store,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Yoga Studio
101,Etobicoke,-79.588437,2.0,Venue Category_Grocery Store,Venue Category_Fast Food Restaurant,Venue Category_Video Store,Venue Category_Fried Chicken Joint,Venue Category_Sandwich Place,Venue Category_Beer Store,Venue Category_Pizza Place,Venue Category_Pharmacy,Venue Category_Ethiopian Restaurant,Venue Category_Empanada Restaurant


In [34]:
x_merged.loc[x_merged['Cluster Labels'] == 3, x_merged.columns[[1] + list(range(5, x_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,North York,-79.565963,3.0,Venue Category_Empanada Restaurant,Venue Category_Pizza Place,Venue Category_Electronics Store,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Eastern European Restaurant,Venue Category_Yoga Studio


In [35]:
x_merged.loc[x_merged['Cluster Labels'] == 4, x_merged.columns[[1] + list(range(5, x_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,-79.374714,4.0,Venue Category_Cafeteria,Venue Category_Yoga Studio,Venue Category_Eastern European Restaurant,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Electronics Store,Venue Category_Colombian Restaurant
